## Import libraries

In [1]:
import json
import time
import os

import pandas as pd
import numpy as np
from geopy.geocoders import Nominatim
import folium
import openrouteservice

## Read Journal Log csv file

In [2]:
pdfLogs = pd.read_csv("logs/journal-log_2022-12-17.csv")
pdfLogs = pdfLogs[pdfLogs.Started>"2022-12-17"]

## Geo Decode addresses
Notice that Nominatim could not decode some of the addresses and I addded them manually to json file.

In [3]:
ADDRESS_MAP_FILE = "cache/addressMap.json"

if os.path.isfile(ADDRESS_MAP_FILE):
    with open(ADDRESS_MAP_FILE, "r") as fp:
        addMap = json.load(fp) 
    print(f"loaded from cache folder: {len(addMap)} mappings")
else:
    address = pdfLogs[["Start address","End address"]].values
    address = [item for sublist in address for item in sublist]
    address = list(set(address))
    print(len(address))
    geolocator = Nominatim(user_agent = "LocationMappe")
    addMap={}
    for a in address:
        location = geolocator.geocode(a)
        if location:
            addMap.update({a: {"lat":location.latitude,"lon":location.longitude}})
    
    # save the mapping as json file
    with open(ADDRESS_MAP_FILE, "w") as fp:
        json.dump(addMap , fp)
    print(f"queried with Nominatim and saved in cache folder: {len(addMap)} mappings")

loaded from cache folder: 48 mappings


## Decode the start & end addresses to Lat/Lon with the mapping table
- Caching the results to routesMap.json file
- setup openrouteservice client with api key, you can signup https://openrouteservice.org 
- After signup, you can see your API key available under the dashboard tab.

In [4]:
def decodeAddressesToCoordinates(pdfin):
    pdfin['StartAddressCoordinates'] = pdfin['Start address'].map(addMap)
    pdfin = (pdfin
             .join(pd
                   .json_normalize(pdfin.StartAddressCoordinates)
                   .rename(columns={"lat":"StartLat","lon":"StartLon"})
                  )
             .drop('StartAddressCoordinates', axis='columns')
            )
    pdfin['EndAddressCoordinates'] = pdfin['End address'].map(addMap)
    pdfin = (pdfin
             .join(pd
                   .json_normalize(pdfin.EndAddressCoordinates)
                   .rename(columns={"lat":"EndLat","lon":"EndLon"})
                  )
             .drop('EndAddressCoordinates', axis='columns')
            )
    return pdfin

In [5]:
ROUTES_MAP_FILE = "cache/routesMap.json"

pdfRoutes = pdfLogs.loc[:,['Started', 'Start address', 'End address']]
pdfRoutes = decodeAddressesToCoordinates(pdfRoutes)

if os.path.isfile(ROUTES_MAP_FILE):
    pdfRoutesMap = pd.read_json(ROUTES_MAP_FILE).set_index("Started")
    pdfRoutes = pdfRoutes.set_index("Started").join(pdfRoutesMap).reset_index().replace(np.nan, None)
    print(f"loaded from cache folder: {pdfRoutesMap.shape[0]} mappings")
else:
    pdfRoutes["route"] = None

openrouteserviceAPIKey='xxxxxx'
exportRoutesMap = False
routes = pdfRoutes["route"].values
for idx in range(len(routes)):
    if routes[idx]==None:
        exportRoutesMap = True
        print(idx,end=",")
        drivePath = pdfRoutes.iloc[idx]
        
        # putting some pause while we use the free API call
        time.sleep(2)
        
        # prepare the client
        client = openrouteservice.Client(key=openrouteserviceAPIKey)
        
        # set routing start and end coordinates
        coords = ((       drivePath["StartLon"],drivePath["StartLat"] ),(       drivePath["EndLon"],drivePath["EndLat"]    )) #
        
        # just an expception for the journey starting from Pisa, to take a route that goes through (10.722359,43.333864)
        if "Pisa" in drivePath["Start address"]:
            coords = (coords[0] , (10.722359,43.333864), coords[1])
        
        #call the API
        res = client.directions(coords, optimize_waypoints=True, profile='driving-car')
        geometry = client.directions(coords)['routes'][0]['geometry']
        decoded = openrouteservice.convert.decode_polyline(geometry)
        routes[idx]=decoded
        
if exportRoutesMap:
    pdfRoutesMap = pdfRoutes.loc[:,["Started","route"]].dropna()
    pdfRoutesMap.to_json(ROUTES_MAP_FILE, orient="records")#.set_index("Started")
print("done.")

loaded from cache folder: 51 mappings
done.


In [6]:
pdfRoutesMap = pd.read_json(ROUTES_MAP_FILE).set_index("Started")
pdfLogs = decodeAddressesToCoordinates(pdfLogs)
pdfLogs = pdfLogs.set_index("Started").join(pdfRoutesMap).reset_index().replace(np.nan, None)

# Import Fill-Ups

In [7]:
pdfFillup = pd.read_csv("logs/Fuelup_Records_2022-12-17.csv")
pdfFillup = pdfFillup[pdfFillup.Vehicle=="CoCo"]
pdfFillup = pdfFillup[pdfFillup.Type=="Gas"]
pdfFillup = pdfFillup[pdfFillup.Date>="2022-12-17"]
pdfFillup = pdfFillup[pdfFillup.Date<="2022-12-25"]
pdfFillup["Tags"] = pdfFillup["Tags"].fillna("Gas")
pdfFillup.Tags = pdfFillup.Tags.str.rstrip()
pdfFillup["Electric"] = pdfFillup["Notes"].str.replace("kWh","").astype("float")
pdfFillup["Cost/Liter"] = pdfFillup["Cost/Liter"].str.replace("CHF ","").astype("float")
pdfFillup["Total Cost"] = pdfFillup["Total Cost"].str.replace("CHF ","").astype("float")
pdfFillup.loc[pdfFillup.Tags=="Electric","Liters"]=pdfFillup.loc[pdfFillup.Tags=="Electric","Electric"]
pdfFillup.loc[pdfFillup.Tags=="Electric","Cost/Liter"]=(pdfFillup.loc[pdfFillup.Tags=="Electric","Total Cost"]
                                                        /pdfFillup.loc[pdfFillup.Tags=="Electric","Electric"]
                                                       )
pdfFillup["kWh"] = pdfFillup["Liters"]
pdfFillup.loc[pdfFillup.Tags=="Electric","Liters"]=None
pdfFillup.loc[pdfFillup.Tags!="Electric","kWh"]=None
pdfFillup.Odometer = pdfFillup.Odometer.str.replace("'","").astype("int")
pdfFillup = pdfFillup.drop(["Type", "Vehicle","Location","Gas Brand","Payment Type","Tire Pressure","Notes","Electric","Tags"],axis="columns")
pdfFillup = pdfFillup.rename(columns={"Cost/Liter":"Cost/unit","Services":"Notes"})

print(f"Total Electric Filleups: \t{pdfFillup.kWh.sum():.1f} kWh, \t{pdfFillup.kWh.count():.1f} times, \t{pdfFillup.loc[:,['Total Cost','kWh']].dropna()['Total Cost'].sum():.1f} CHF")
print(f"Total Gas Filleups: \t\t{pdfFillup.Liters.sum():.1f} Liter, \t{pdfFillup.Liters.count():.1f} times, \t{pdfFillup.loc[:,['Total Cost','Liters']].dropna()['Total Cost'].sum():.1f} CHF")

Total Electric Filleups: 	89.2 kWh, 	13.0 times, 	19.2 CHF
Total Gas Filleups: 		115.8 Liter, 	4.0 times, 	167.2 CHF


## Load routes and join with fillups

In [8]:
pdfLogs.loc[pdfLogs["Distance (km)"]<1,"End odometer (km)"]=pdfLogs.loc[pdfLogs["Distance (km)"]<1,"Start odometer (km)"] + pdfLogs.loc[pdfLogs["Distance (km)"]<1,"Distance (km)"]
pdf = pdfLogs.set_index("End odometer (km)").join(pdfFillup.drop(["L/100km","Date","Time","Filled Up"],axis="columns")
                                              .rename(columns={"Odometer":"End odometer (km)","Liters":"Fillup (Liter)", "kWh":"Fillup (kWh)"}).set_index("End odometer (km)"), 
         how="left", rsuffix="_r"
        ).reset_index()
pdf.loc[:,["End odometer (km)","Start odometer (km)","Fillup (kWh)","Fillup (Liter)", "Distance (km)"]]
cols=["Battery regeneration (kWh)","Battery consumption (kWh)","Fuel consumption (litres)","Distance (km)","Fillup (Liter)","Fillup (kWh)"]
printStr=[]
for colName in cols:
    printStr.append([colName,f"{pdf[colName].sum():.1f}"])
display(pd
        .DataFrame(printStr,
                   columns=["Aggregation","Value"])
        .set_index("Aggregation")
       )

,Value
Aggregation,
Battery regeneration (kWh),90.1
Battery consumption (kWh),160.7
Fuel consumption (litres),120.3
Distance (km),2077.5
Fillup (Liter),115.8
Fillup (kWh),89.2


In [9]:
# set random factor for duplicated end markers
pdf["randFactor"]=0
pdf["Remarks"]=None
remarks = pdf["Remarks"]
idx = pdf[pdf.loc[:,["EndLat","EndLon"]].duplicated(keep=False)].index
pdf.loc[idx,"randFactor"]=0.00066

pdf.sort_values(by=['Fillup (kWh)',"Notes","Fillup (Liter)"], ascending=True, inplace=True)

m = folium.Map(location=[45.500834101180175, 9.771564151850093], #(pdf.StartLat.mean()+pdf.EndLat.mean())/2, (pdf.StartLon.mean()+pdf.EndLon.mean())/2]
               zoom_start=7)
folium.LatLngPopup().add_to(m)
counts = {"Park&Charge":0,
          "Missing&ChargeMalfunction":0,
         }
for idx,drivePath in pdf.iterrows():
    LinesCoords = [[i[1],i[0]]for i in drivePath["route"]["coordinates"]]
    tooltip = "Click here for more details"
    textStr = ""
    if pd.notna(drivePath['Fillup (kWh)']):
        icon=folium.Icon(color='green', prefix='fa',icon='plug')
        counts["Park&Charge"] += 1
        pdf.loc[idx,"Remarks"]="Park&Charge"
    elif pd.notna(drivePath['Notes']):
        icon=folium.Icon(color='red', prefix='fa',icon='plug')
        counts["Missing&ChargeMalfunction"] += 1
        pdf.loc[idx,"Remarks"]="Missing&ChargeMalfunction"
    elif pd.notna(drivePath['Fillup (Liter)']):
        icon=folium.Icon(color='blue', prefix='fa',icon='tint')
    else:
        icon=folium.Icon(color='black', prefix='fa',icon='parking')
        
    if pd.notna(drivePath['Notes']):
        textStr += "Notes:"+str(drivePath['Notes'])+"<br>"    
    if pd.notna(drivePath['Fillup (kWh)']):
        textStr += str(drivePath['Fillup (kWh)'])+" (+kWh fillup)"+"<br>"   
    if pd.notna(drivePath['Fillup (Liter)']):
        textStr += str(drivePath['Fillup (Liter)'])+" (+liter fillup)"+"<br>"  
        
    textStr += (
                ""
                +"Odometer:"+str(drivePath['End odometer (km)'])+"<br>"    
                +"Stopped:"+str(drivePath['Stopped'])+"<br>"    
               )
    
    iframe = folium.IFrame(textStr)
    popup = folium.Popup(iframe, min_width=300, max_width=300)
    
    folium.PolyLine(LinesCoords, color="blue",opacity=0.7).add_to(m)
    
    
    folium.Marker(location=(drivePath["EndLat"]+(np.random.rand()-0.5)*drivePath["randFactor"],drivePath["EndLon"]+(np.random.rand()-0.5)*drivePath["randFactor"]),
                  popup=popup, icon=icon, tooltip = tooltip).add_to(m)
m.save("./output/index.html")

In [10]:
totalDist = 2077.5
theoreticalElecDistance = 89.2/0.22 # Total charging kWh over average kWh/km
pdf["Electric"]=pd.notna(pdf['Fillup (kWh)'])
pdfAgg = pdf.loc[:,["Electric","Total Cost",'Fillup (kWh)','Fillup (Liter)']].groupby("Electric").sum()
pdfAgg["Conumption/100km"] = pdfAgg['Fillup (kWh)']/(totalDist/100) + pdfAgg['Fillup (Liter)']/20.775
pdfAgg["Cost/100km"] = pdfAgg['Total Cost']/(totalDist/100)
pdfAgg["Cost Share"] = pdfAgg['Total Cost']/pdfAgg['Total Cost'].sum()
pdfAgg["km Share"] = np.array([totalDist-theoreticalElecDistance, theoreticalElecDistance])/totalDist
pdfAgg["km Share with Regen."] = np.array([totalDist-theoreticalElecDistance*2, theoreticalElecDistance*2])/totalDist
pdfAgg["cost/100km split"] = pdfAgg['Total Cost']/(pdfAgg["km Share"]*totalDist)
pdfAgg["cost/100km with Regen. split"] = pdfAgg['Total Cost']/(pdfAgg["km Share with Regen."]*totalDist)
pdfAgg["Conumption/100km split"] = (pdfAgg['Fillup (Liter)']+pdfAgg['Fillup (kWh)'])/(pdfAgg["km Share"]*totalDist/100)
pdfAgg["Conumption/100km with Regen. split"] = (pdfAgg['Fillup (Liter)']+pdfAgg['Fillup (kWh)'])/(pdfAgg["km Share with Regen."]*totalDist/100)

print(f"Count of Park&Charge: \t\t\t{counts['Park&Charge']}")
print(f"Count of Missing & Charge Malfunction: \t{counts['Missing&ChargeMalfunction']}")
display(pdfAgg)

Count of Park&Charge: 			13
Count of Missing & Charge Malfunction: 	9


,Total Cost,Fillup (kWh),Fillup (Liter),Conumption/100km,Cost/100km,Cost Share,km Share,km Share with Regen.,cost/100km split,cost/100km with Regen. split,Conumption/100km split,Conumption/100km with Regen. split
Electric,,,,,,,,,,,,
False,167.25,0.0,115.829,5.575403,8.050542,0.897168,0.804835,0.609671,0.100027,0.132047,6.927383,9.144942
True,19.17,89.2,0.000,4.293622,0.922744,0.102832,0.195165,0.390329,0.047280,0.023640,22.000000,11.000000


In [11]:
pdf.Started = pd.to_datetime(pdf.Started)
pdf["date"] = pdf.Started.dt.date
pdf["Charge Count"] = pdf["Remarks"]=="Park&Charge"
pdf["Can't Charge Count"] = pdf["Remarks"]=="Missing&ChargeMalfunction"
display(pdf.loc[:,["date","Distance (km)","Fillup (kWh)","Battery regeneration (kWh)","Charge Count","Can't Charge Count"]]
        .groupby(["date"]).sum()
       )

,Distance (km),Fillup (kWh),Battery regeneration (kWh),Charge Count,Can't Charge Count
date,,,,,
2022-12-17,427.0,13.9,8.6,2,0
2022-12-18,46.6,10.0,1.1,2,0
2022-12-19,309.9,9.0,25.1,1,1
2022-12-20,136.6,16.8,7.7,2,1
2022-12-21,108.0,12.0,5.2,2,1
2022-12-22,364.6,3.4,18.1,1,4
2022-12-23,149.6,15.0,3.8,2,1
2022-12-24,533.1,0.0,20.2,0,1
2022-12-25,2.1,9.1,0.3,1,0
